# Profiling

In this lab we will go through how to quickly profile and optimize some code. This lab should have the following file structure:

```bash
Lab 3
├── profiling.ipynb
├── requirements.txt
├── data
    ├── B_mat.npy
    ├── C_mat.npy
    └── cat.mat
├── small_world_propensity
    ├── __init__.py
    └── small_world_propensity.py
└── tests
    ├── __init__.py
    └── test_small_world.py
```

The code is simply a small library that takes a connectivity matrix, and calculates how small-world the associated graph is.

Along the way, it must calculate things like degree, clustering, and path lengths. Let's just run it on some test data to begin with. Create a virtual environment and install the requirements.

In [1]:
import small_world_propensity as swp

import scipy.io as sio
import numpy as np

In [2]:
cat = sio.loadmat('data/cat.mat')['CIJctx']
cat.shape

(52, 52)

This is a connectivity matrix representing functional connectivity between 52 regions in a cat brain. As far as I am aware, the cat was not harmed while collecting this data.

In [3]:
np.random.seed(1337)

swp.small_world_propensity(cat)

,Network C,Network L,ΔC,ΔL,SWP,α,δ,Regular C,Random C,Regular L,Random L
0,0.270482,3.566516,0.169904,0.251567,0.785345,0.976783,0.243679,0.296984,0.141,5.889819,2.785596


OK, so that was super fast. Now lets load in a connectivity matrix representing connections in a resistive switching array.

In [4]:
C = np.load('data/C_mat.npy')
C.shape

(805, 805)

So this is quite a bit bigger! Let's see how it runs...

In [5]:
np.random.seed(1337)

swp.small_world_propensity(C)

,Network C,Network L,ΔC,ΔL,SWP,α,δ,Regular C,Random C,Regular L,Random L
0,0.042583,11.987697,0.0,0.232254,0.835772,1.570796,1.0,0.04078,0.039689,26.247016,7.67405


12s might not seem a lot, but suppose you want to generate some data on how SWP changes with size. In order to do this, you'll want to try a range of size, say 5,000. And since SWP contains an element of stochasticity, you'll need to run each network to get a mean and variance, so maybe 100 different instances. That's 500,000 runs.

It's going to take you well over 1,500 hours to run all of these networks! That's preposterous. If you have an 8 core processor you can cut this down to 200 hours (or about 8 days), with parallization. Assuming you don't mind thrashing your Macbook Pro M1...

## cProfile
Let's use cProfile and try and find where we are spending time. `cat` is too small and `C` is too big, so we try an intermediate network instead.

In [8]:
import cProfile

In [6]:
B = np.load('data/B_mat.npy')
B.shape

(361, 361)

In [9]:
cProfile.run('swp.small_world_propensity(B)', sort='cumtime')

         433921 function calls (433898 primitive calls) in 1.417 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.417    1.417 {built-in method builtins.exec}
        1    0.000    0.000    1.417    1.417 <string>:1(<module>)
        1    0.000    0.000    1.417    1.417 small_world_propensity.py:15(small_world_propensity)
        1    0.002    0.002    1.417    1.417 small_world_propensity.py:40(_small_world_propensity)
        3    0.003    0.001    0.698    0.233 small_world_propensity.py:146(get_average_paths)
        3    0.679    0.226    0.679    0.226 {method 'average_path_length' of 'igraph._igraph.GraphBase' objects}
        1    0.127    0.127    0.669    0.669 small_world_propensity.py:201(regular_matrix_generator)
   429440    0.532    0.000    0.532    0.000 {method 'randint' of 'numpy.random.mtrand.RandomState' objects}
        3    0.004    0.001    0.025    0.008 small_worl

So how do we interpret the above? We have sorted by cumulative time. The top 4 entries are essentially just calls to the main function, telling us how long we spend inside `small_world_propensity`. Next we can see that `get_average_paths` is taking an awful long time! We are spending just over half of our time inside this function! What's next on the list? generating the random integers is taking up most of the rest of the time, but we can't really avoid this. Perhaps there are more efficient random number generators that we can use?

Let's start with the path length. Let's look at the `get_average_paths` function:

```python
def get_average_paths(W: np.ndarray) -> float:
    """Get the average shortest path length of W

    Args:
        W (np.ndarray): Symmetric adjacency matrix

    Returns:
        float: Average shortest path lengths.
    """
    igG = ig.Graph.Weighted_Adjacency((1/W).tolist(), mode="UNDIRECTED")
    L_W = igG.average_path_length(weights="weight")

    return L_W
```

The details aren't really important, but we are converting our matrix into a graph using the `igraph` library, and then using a method from that library to get the path lengths. In our research, we discover that `scipy` has a sparse graph library that might be useful. We know that our graphs are sparse, so perhaps this will be more efficient...

We can rewrite our function as follows:

```python
def get_average_paths(W: np.ndarray) -> float:
    """Get the average shortest path length of W

    Args:
        W (np.ndarray): Symmetric adjacency matrix

    Returns:
        float: Average shortest path lengths.
    """
    path_matrix = csgraph.shortest_path(1/W, directed=False, unweighted=False)
    L_W = np.triu(path_matrix).sum() / (len(W) * (len(W) - 1) / 2)

    return L_W
```

Now we make the substitution. This also means that we now no longer need to include the weighty `igraph` package, so we take that out as well, and rerun the tests...

All tests pass, so now we can try profiling again. Remember, you will need to restart the kernel.

In [1]:
import small_world_propensity as swp

import scipy.io as sio
import numpy as np
import cProfile

B = np.load('data/B_mat.npy')
B.shape

(361, 361)

In [2]:
cProfile.run('swp.small_world_propensity(B)', sort='cumtime')

         432156 function calls (432041 primitive calls) in 0.817 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.817    0.817 {built-in method builtins.exec}
        1    0.000    0.000    0.817    0.817 <string>:1(<module>)
        1    0.000    0.000    0.817    0.817 small_world_propensity.py:13(small_world_propensity)
        1    0.000    0.000    0.817    0.817 small_world_propensity.py:38(_small_world_propensity)
        1    0.136    0.136    0.608    0.608 small_world_propensity.py:199(regular_matrix_generator)
   423544    0.462    0.000    0.462    0.000 {method 'integers' of 'numpy.random._generator.Generator' objects}
        3    0.000    0.000    0.165    0.055 small_world_propensity.py:144(get_average_paths)
        3    0.148    0.049    0.165    0.055 {scipy.sparse.csgraph._shortest_path.shortest_path}
        1    0.021    0.021    0.023    0.023 small_world_propensity.p

In [3]:
C = np.load('data/C_mat.npy')
swp.small_world_propensity(C)

,Network C,Network L,ΔC,ΔL,SWP,α,δ,Regular C,Random C,Regular L,Random L
0,0.042583,11.987697,0.0,0.232229,0.83579,1.570796,1.0,0.04078,0.039691,26.247026,7.674664


Great! We have cut the time to around 45% of the original. Let's see if we can do something about the randomness...we can use the magic function called timeit to compare simple function calls.

In [19]:
%%timeit
np.random.randint(0,10000)

1.28 µs ± 5.06 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


The default random number generator is PCG64, which is quite fast. But as we talked about in the first Lab, this method isn't the preferred way of generating random numbers. So let's try using the `Generator` class and the `SFC64` algorithm.

In [21]:
from numpy.random import Generator, PCG64, SFC64

rngPC = Generator(PCG64(1337))
rngSF = Generator(SFC64(1337))


In [22]:
%%timeit
rngPC.integers(0,10000)

1.05 µs ± 5.94 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [24]:
%%timeit
rngSF.integers(0,10000)

1.03 µs ± 3.82 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


So this method is a bit faster, with `SFC64` giving trivially better performance. So we'll make an update to our function, and also make sure that we can pass a seed so that we can iterate over many instances.

In [1]:
import small_world_propensity as swp
import numpy as np

In [2]:
C = np.load('data/C_mat.npy')
swp.small_world_propensity(C, seed=1337)


,Network C,Network L,ΔC,ΔL,SWP,α,δ,Regular C,Random C,Regular L,Random L
0,0.042583,11.987697,0.0,0.232073,0.8359,1.570796,1.0,0.04078,0.039689,26.247057,7.67842


In [7]:
cat = sio.loadmat('data/cat.mat')['CIJctx']

swp.small_world_propensity(cat, 1337)

,Network C,Network L,ΔC,ΔL,SWP,α,δ,Regular C,Random C,Regular L,Random L
0,0.270482,3.566516,0.170363,0.247591,0.787485,0.968113,0.232639,0.29693,0.141686,5.874811,2.806938


So not really a whole lot of gain, but now the user at least has more control over seeding the random number generator. If other researchers want to replicate your code, then they can simply use the same initial seed.